In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager  # 크롬 드라이버 최신버전 관리
import time

service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')
options.add_argument('--start-maximized')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36')
options.add_argument('--lang=ko_KR')

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=A")
time.sleep(3)

search_button = driver.find_element(By.CLASS_NAME, "sc-dzagxs-2")
search_button.click()
time.sleep(2)

popular_keyword = driver.find_elements(By.CSS_SELECTOR, ".search-home-popular-wrap .sc-1xzm8bl-1")
target_keyword = popular_keyword[2].text
print("목표 키워드 : ", target_keyword)

search_input = driver.find_element(By.CSS_SELECTOR, ".search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
time.sleep(1)

search_input.send_keys(Keys.RETURN)
time.sleep(5)

print("검색완료. 현재 페이지 타이틀 : ", driver.title)
driver.quit()

목표 키워드 :  반팔
검색완료. 현재 페이지 타이틀 :  반팔 | 무신사 추천 브랜드


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager  
import pandas as pd
import time
import re


service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')
options.add_argument('--start-maximized')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36')
options.add_argument('--lang=ko_KR')

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=A")
time.sleep(3)

search_button = driver.find_element(By.CLASS_NAME, "sc-dzagxs-2")
search_button.click()
time.sleep(2)

popular_keyword = driver.find_elements(By.CSS_SELECTOR, ".search-home-popular-wrap .sc-1xzm8bl-1")
target_keyword = popular_keyword[2].text
print("목표 키워드 : ", target_keyword)

search_input = driver.find_element(By.CSS_SELECTOR, ".search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
time.sleep(1)

search_input.send_keys(Keys.RETURN)
time.sleep(5)

print("검색완료. 현재 페이지 타이틀 : ", driver.title)

scroll_pulse = 2
for _ in range(10) :
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pulse)

products = driver.find_elements(By.CSS_SELECTOR, "div.sc-d36st-1.elXjMR")[:100]
product_data = []

for product in products :
    try : 
        name = product.find_element(By.CSS_SELECTOR, "div.sc-cMuefe.jIlLeV a.gtm-select-item span").text.strip()
        url = product.find_element(By.CSS_SELECTOR, "div.sc-cMuefe.jIlLeV a.gtm-select-item").get_attribute("href")
        review_info = product.find_elements(By.CSS_SELECTOR, "span.text-etc_11px_reg.text-yellow.font-pretendard")[1].text.strip()
        review_count = int(re.sub(r"[^\d]", "", review_info)) if review_info else 0
        product_data.append({"상품명" : name, "URL" : url, "리뷰수" : review_count})
    except :
        continue

df = pd.DataFrame(product_data)
df_top30 = df.sort_values(by="리뷰수", ascending=False).head(30).reset_index(drop=True)

driver.quit()
df_top30.head()


목표 키워드 :  반팔
검색완료. 현재 페이지 타이틀 :  반팔 | 무신사 추천 상품


,상품명,URL,리뷰수
0,봉봉이 레귤러 or 오버 티셔츠 2colors,https://www.musinsa.com/products/4010800,821
1,먼데이루틴 브로콜리 반팔티셔츠_화이트,https://www.musinsa.com/products/2463159,656
2,냥이가 지켜보고 있다 티셔츠 [화이트],https://www.musinsa.com/products/3203072,500
3,HIDE AWAY T-SHIRT in D/Grey VW2ME124-13,https://www.musinsa.com/products/2568914,461
4,얼터네이트 스트라이프 티 블루,https://www.musinsa.com/products/4135354,426


In [3]:
from openpyxl import Workbook

driver = webdriver.Chrome(service=service, options=options)
all_reviews = []
for idx, row in df_top30.iterrows():
    print(f"{idx+1}번째 상품리뷰 수집 중:, {row['상품명']}")
    driver.get(row['URL'])
    time.sleep(2)

    try :
        review_tab=driver.find_element(By.XPATH, "//button[@data-button-id='prd_review_tab']")
        review_tab.click()
        time.sleep(2)

        for _ in range(3) :
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)
            
        review_elements = driver.find_elements(By.CSS_SELECTOR, ".text-body_13px_reg.text-black.font-pretendard")
        for review in review_elements[:10] :
            text = review.text.strip()
            if text :
                all_reviews.append({
                    "상품명" : row["상품명"],
                    "리뷰" : text
                })
    except Exception as e:
        print("리뷰 수집 실패: ", e)
        continue

driver.quit()

wb = Workbook()
ws = wb.active
ws.title= "무신사 리뷰"

ws.append(["상품명", "리뷰"])

for r in all_reviews :
    ws.append([r["상품명"],r["리뷰"]])

wb.save("musinsa_top30_reviews.xlsx")
print("✅엑셀 저장 완료 : musinsa_top30_reviews.xlsx")

1번째 상품리뷰 수집 중:, 봉봉이 레귤러 or 오버 티셔츠 2colors
2번째 상품리뷰 수집 중:, 먼데이루틴 브로콜리 반팔티셔츠_화이트
3번째 상품리뷰 수집 중:, 냥이가 지켜보고 있다 티셔츠 [화이트]
4번째 상품리뷰 수집 중:, HIDE AWAY T-SHIRT in D/Grey VW2ME124-13
5번째 상품리뷰 수집 중:, 얼터네이트 스트라이프 티 블루
6번째 상품리뷰 수집 중:, MOT SERIES TEE(RIDE THE COAST)_OFF WHITE
7번째 상품리뷰 수집 중:, [20수] HAPPY PUPPY 반팔티 화이트
8번째 상품리뷰 수집 중:, 링거송티
9번째 상품리뷰 수집 중:, 키튼 패치 피그먼트 반팔티 브라운
10번째 상품리뷰 수집 중:, 15MIN 쿼터 팀플레이 그래픽 로고 반팔티 화이트 블루
11번째 상품리뷰 수집 중:, 루즈핏 스트라이프 반팔티 ( 4Color )
12번째 상품리뷰 수집 중:, [노티카 x 인세인개러지] 그래픽 자수 티셔츠
13번째 상품리뷰 수집 중:, EL 데이지 ver2 반팔티셔츠 4Color
14번째 상품리뷰 수집 중:, 럭비 메쉬 져지 반팔 티셔츠_화이트
15번째 상품리뷰 수집 중:, ICE LITE 초냉감 세미오버핏 티셔츠
16번째 상품리뷰 수집 중:, 우먼즈 B.LIFE 오버핏 반팔티
17번째 상품리뷰 수집 중:, Renee Square Neck Slim T-Shirt - 5COL
18번째 상품리뷰 수집 중:, 오버핏 래글런 트랙 라인 반팔 티셔츠 스타로고 버건디
19번째 상품리뷰 수집 중:, [2PACK] 소프트 보트넥 7부 숏 슬리브 (10 Color)
20번째 상품리뷰 수집 중:, VINTAGE ARTWORK TEE_burgundy
21번째 상품리뷰 수집 중:, [반팔ver]나그랑뒷반팔티셔츠(PBE03S-WHTW004)
22번째 상품리뷰 수집 중:, Big Logo Compil 그래픽 티셔츠 - 8 컬러
23번째 상품리뷰 수집 중:, 씬 스트라이프 롤 업 포켓 헨리넥 반팔 티셔츠_블랙
24번째 

In [19]:
pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

EXCEL_PATH = "musinsa_top30_reviews.xlsx"
DB_NAME = "musinsa_db_v3"
TABLE_NAME = "reviews"
USER = "root"
PASSWORD = "753951DnDn!!"
HOST = "127.0.0.1"
PORT = 3306

df = pd.read_excel(EXCEL_PATH)

conn = pymysql.connect(host=HOST, user=USER, password=PASSWORD, port=PORT)
cursor = conn.cursor()
cursor.execute(
    f"CREATE DATABASE IF NOT EXISTS {DB_NAME} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"
)
conn.close()

engine_str = f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}?charset=utf8mb4"
engine = create_engine(engine_str)

df.to_sql(name=TABLE_NAME, con=engine, if_exists="replace", index=False)
print(f"✅데이터가 MySQL [{DB_NAME}]의 테이블 [{TABLE_NAME}에 성공적으로 저장되었습니다.]")

✅데이터가 MySQL [musinsa_db_v3]의 테이블 [reviews에 성공적으로 저장되었습니다.]


In [5]:
pip install gspread

Defaulting to user installation because normal site-packages is not writeable

   ---------------------------------------- 0/6 [rsa]
   ---------------------------------------- 0/6 [rsa]
   ------ --------------------------------- 1/6 [oauthlib]
   ------ --------------------------------- 1/6 [oauthlib]
   ------ --------------------------------- 1/6 [oauthlib]
   ------ --------------------------------- 1/6 [oauthlib]
   ------------- -------------------------- 2/6 [requests-oauthlib]
   -------------------- ------------------- 3/6 [google-auth]
   -------------------- ------------------- 3/6 [google-auth]
   -------------------- ------------------- 3/6 [google-auth]
   -------------------- ------------------- 3/6 [google-auth]
   -------------------- ------------------- 3/6 [google-auth]
   --------------------------------- ------ 5/6 [gspread]
   ---------------------------------------- 6/6 [gspread]

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [7]:
pip install oauth2client

Defaulting to user installation because normal site-packages is not writeable

   -------------------- ------------------- 1/2 [oauth2client]
   -------------------- ------------------- 1/2 [oauth2client]
   ---------------------------------------- 2/2 [oauth2client]

Note: you may need to restart the kernel to use updated packages.


In [10]:
from konlpy.tag import Okt
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd

df = pd.read_excel("musinsa_top30_reviews.xlsx")

okt = Okt()
df["리뷰_키워드"] = df["리뷰"].apply(lambda x : ", ".join(okt.nouns(str(x))))

scope = ["https://spreadsheets.google.com/feeds" , "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("instant-ascent-468703-h8-ab4b106c6a3c.json", scope)
client = gspread.authorize(creds)

spreadsheet = client.open("무신사 리뷰 분석_v1")
sheet1 = spreadsheet.sheet1
sheet1.clear()
sheet1.update([df.columns.values.tolist()] + df.values.tolist())

try :
    spreadsheet.add_worksheet(title="키워드 분석", rows="1000", cols="20")
except : 
    pass

sheet2 = spreadsheet.worksheet("키워드 분석")
sheet2.clear()
sheet2.update([["상품명", "리뷰", "리뷰_키워드"]] + df[["상품명", "리뷰", "리뷰_키워드"]].values.tolist())
             
print("✅ 구글 스프레드 시트에 원본 저장  + 키워드 분석 저장 안료!")

✅ 구글 스프레드 시트에 원본 저장  + 키워드 분석 저장 안료!
